In [ ]:
!pip install -q faiss-cpu sentence-transformers python-docx pymupdf pandas openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 821.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import os
import fitz  # PyMuPDF
import docx
import pandas as pd

def load_file(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == '.pdf':
        return extract_text_from_pdf(path)
    elif ext == '.docx':
        return extract_text_from_docx(path)
    elif ext in ['.csv', '.xls', '.xlsx']:
        return extract_text_from_csv_xls(path)
    else:
        raise ValueError("Unsupported file type: " + ext)

def extract_text_from_pdf(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_text_from_docx(path):
    doc = docx.Document(path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_csv_xls(path):
    df = pd.read_excel(path) if path.endswith(('xls', 'xlsx')) else pd.read_csv(path)
    return df.to_string(index=False)

uploaded = files.upload()
file_path = next(iter(uploaded))
text = load_file(file_path)
print(text[:1000])

Saving the-alchemist-paulo-coelho.pdf to the-alchemist-paulo-coelho.pdf
 
 
 
The alchemist
 
 
By
 
Paulo Coelho
 
 
 
TEN YEARS ON
 
I REMEMBER RECEIVING A LETTER FROM THE AMERICAN publisher Harper
Collins that said that: “reading The Alchemist was like getting up at dawn and seeing the sun
rise while the rest of the world still slept.” I went outside, looked up at the sky, and thought to
myself: “So, the book is going to be published in English!” At the time, I was struggling to
establish myself as a writer and to follow my path despite all the voices telling me it was
impossible.
And little by little, my dream was becoming reality. Ten, a hundred, a thousand, a million
copies sold in America. One day, a Brazilian journalist phoned to say that President Clinton
had been photographed reading the book. Some time later, when I was in Turkey, I opened the
magazine Vanity Fair and there was Julia Roberts declaring that she adored the book. Walking
alone down a street in Miami, I heard a 

In [ ]:
def chunk_text(text, chunk_size=700, overlap=160):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(text)
print(f"Total chunks: {len(chunks)}")

Total chunks: 400


In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

def generate_embeddings(chunks):
    return embedder.encode(chunks)

embeddings = generate_embeddings(chunks)

In [ ]:
import faiss
import numpy as np

def create_faiss_index(embeddings, chunks):
    dim = embeddings.shape[1] if hasattr(embeddings, 'shape') else len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, chunks

index, chunk_store = create_faiss_index(embeddings, chunks)

In [ ]:
import numpy as np
from groq import Groq

# Initialize Groq client with your API key
groq_api_key = "Use an LLM API KEY here and also pay attention that its environment has been set"  #USE GROQ API KEY ITS FREE
client = Groq(api_key=groq_api_key)

def search_index(query, index, embedder, chunks, top_k=5):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [chunks[i] for i in indices[0]]

def ask_llm(relevant_chunks, query):
    context = "\n\n".join(relevant_chunks)
    prompt = f"""You are a literary assistant AI. Use only the context provided below to answer the user's question.
If the answer is not in the context, say "I don't know."

Context:
{context}

Question:
{query}

Answer:"""

    response = client.chat.completions.create(
        model="llama3-8b-8192",  # You can use other models like llama3-8b-8192
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,
        max_tokens=700
    )

    return response.choices[0].message.content


In [ ]:
query = "Why did he agreed to work at the shop?"
top_chunks = search_index(query, index, embedder, chunk_store)
answer = ask_llm(top_chunks, query)

print("💬 Answer:\n", answer)

💬 Answer:
 According to the context, the boy agreed to work at the shop because the merchant treated him fairly, and he received a good commission for each piece he sold. Additionally, the boy had already been able to put some money aside and was working towards a goal of buying sheep.
